# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import nltk
import import_ipynb
from challenge_1 import clean_up
from challenge_1 import tokenize
from challenge_1 import stem_and_lemmatize
from challenge_1 import remove_stopwords
import pandas as pd
import numpy as np

importing Jupyter notebook from challenge_1.ipynb


In [2]:
samples_size = 20000
top_words_size = 3000

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(txt)

{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}

In [4]:
tweets = pd.read_csv('../data/Sentiment140.csv')
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
tweets.shape

(1600000, 6)

In [6]:
tweets_sample = tweets.sample(n=samples_size)
tweets_sample.shape

(20000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [7]:
# your code here
tweets_sample['text_processed'] = tweets_sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
tweets_sample['category'] = tweets_sample.target == 4

In [8]:
tweets_sample['category'].value_counts()

False    10016
True      9984
Name: category, dtype: int64

In [9]:
tweets_sample.head(10)

,target,ids,date,flag,user,text,text_processed,category
962816,4,1827389404,Sun May 17 10:36:28 PDT 2009,NO_QUERY,a13pt1runner,@cwoodworth email me at a13pt1runner@gmail.com...,"[cwoodworth, email, aptrunn, gmail, com, want,...",True
952423,4,1824447630,Sun May 17 01:25:30 PDT 2009,NO_QUERY,sweynh,@bbillybilltweet ROFL - I was not confusing yo...,"[bbillybilltweet, rofl, wa, confus, father, ho...",True
1064440,4,1964704424,Fri May 29 14:40:03 PDT 2009,NO_QUERY,PanicPiien,@gfalcone601 how in the world did you find tha...,"[gfalcon, world, find]",True
1000634,4,1880016307,Thu May 21 23:50:21 PDT 2009,NO_QUERY,revolutionflow,@jmac777 I kno i need to hustle.....jus like u...,"[jmac, kno, need, hustl, ju, like, u, everyday...",True
916512,4,1753328518,Sun May 10 01:06:32 PDT 2009,NO_QUERY,humerfred,@solangeknowles www.youtube.com/watch?v=9FtuV3...,"[solangeknowl, www, youtub, com, watch, v, ftu...",True
1110952,4,1972146381,Sat May 30 08:39:20 PDT 2009,NO_QUERY,paulhovey,@mcnurlenj thanks! That wasn't implying that p...,"[mcnurlenj, thank, impli, peopl, complement, t...",True
1214674,4,1989470539,Mon Jun 01 01:39:50 PDT 2009,NO_QUERY,alissaaf,my hair appointment is on wednesday! excitedex...,"[hair, appoint, wednesday, excitedexcitedexcit]",True
307647,0,2000520259,Mon Jun 01 22:18:09 PDT 2009,NO_QUERY,mdsshellfish,@bernasunoit it doesnt let me save link as HA...,"[bernasunoit, doesnt, let, save, link, haha, b...",False
577868,0,2212785968,Wed Jun 17 14:32:02 PDT 2009,NO_QUERY,vbjb123,@jbdance7 soo look the exact ones up again lol...,"[jbdanc, soo, look, exact, one, lol, american,...",False
173597,0,1964096165,Fri May 29 13:43:04 PDT 2009,NO_QUERY,thesickchick,@neoknits -- That's got to be hard. My parents...,"[neoknit, got, hard, parent, apart, marriag, m...",False


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# your code here# your code here
from nltk import FreqDist
import random

In [11]:
words=[]
for text in tweets_sample.text_processed:
    for el in text:
        words.append(el)
        
all_words = FreqDist(words)

In [12]:
top_words = all_words.most_common(top_words_size)

In [13]:
top_words_w = dict(top_words).keys()

In [14]:
print('first 10 words from the top_5000 words')
list(top_words_w)[:10]

first 10 words from the top_5000 words


['go', 'day', 'get', 'wa', 'thi', 'good', 'work', 'love', 'like', 'quot']

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [15]:
documents = list(zip(tweets_sample.text_processed, tweets_sample.category))
random.shuffle(documents)

In [16]:
def find_features(document):
    word = set(document)
    features = {}
    for w in top_words_w:
        features[w] = (w in word)
    return features

In [17]:
feature_set = [(find_features(tweet), cat) for (tweet, cat) in documents]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [18]:
train_set, test_set = feature_set[:int(len(tweets_sample)*0.8)], feature_set[int(len(tweets_sample)*0.8):]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(10)

Most Informative Features
                 congrat = True             True : False  =     20.3 : 1.0
                     sad = True            False : True   =     19.0 : 1.0
                  throat = True            False : True   =     18.4 : 1.0
                 depress = True            False : True   =     17.0 : 1.0
                    sore = True            False : True   =     17.0 : 1.0
                 terribl = True            False : True   =     15.7 : 1.0
                   proud = True             True : False  =     14.3 : 1.0
              disappoint = True            False : True   =     13.7 : 1.0
                   scare = True            False : True   =     13.4 : 1.0
                     www = True             True : False  =     11.2 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [19]:
nltk.classify.accuracy(classifier, test_set)

0.725

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [20]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [21]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [22]:
# your code here